# Thermal Imaging Data Post-Processing

Michael Mommert, Stuttgart University of Applied Sciences, 2024

In this Notebook, we explore the post-processing and analysis of thermal imaging data using the *flyr* module. Based on sample images, we investigate ways to analyse, manipulate, reproject and visualize such data.

In [ ]:
%pip install numpy \
    matplotlib \
    scikit-image \
    flyr

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
import flyr

## Data I/O and Visualization

First, we will retrieve and read in the two sample thermal images using the *flyr* module: 

In [ ]:
# download both sample images
!wget https://github.com/Hochschule-fuer-Technik-Stuttgart/teaching-mommert/blob/main/thermal/thermal_imaging_flyr/img_1.jpg?raw=true -O img_1.jpg
!wget https://github.com/Hochschule-fuer-Technik-Stuttgart/teaching-mommert/blob/main/thermal/thermal_imaging_flyr/img_2.jpg?raw=true -O img_2.jpg

# reading in image 1
flir_path_1 = "img_1.jpg"
thermogram_1 = flyr.unpack(flir_path_1)

# reading in image 2
flir_path_2 = "img_2.jpg"
thermogram_2 = flyr.unpack(flir_path_2)

The thermograms can be converted into temperatures, using different scales. For instance, we can output the temperatures in units of Kelvins: 

In [ ]:
t = thermogram_1.kelvin
t.shape, t.min(), t.mean(), t.max()

... and degrees Celsius...

In [ ]:
t = thermogram_1.celsius
t.shape, t.min(), t.mean(), t.max()

The temperature conversion uses a set of parameters, that are encoded as metadata in the underlying `jpg` files. We can access these parameters using

In [ ]:
thermogram_1.metadata

These parameters can be modified, for instance, to use a different emissivity. We will investigate this below.

For now, let's plot the temperature array. Before we do so, we define a suitable colormap. Any of the colormaps listed [here](https://matplotlib.org/stable/gallery/color/colormap_reference.html) can be used. 

In [ ]:
cmap = 'rainbow'  # define colormap, feel free to try different colormaps styles

im = plt.imshow(t, cmap=cmap)  # plot the temperature array
plt.colorbar(im, label='Temperature (Celsius)')  # add a corresponding colorbar

The image shows a thermogram of a plastered brick wall with different circular and rectangular signal markers attached. We will see below that the second image shows the same wall, taken from a different angle.

Let's explore how we can alter the visualization of the data. Using the keyword arguments `vmin` and `vmax` of the `imshow` function, we can modify the range of values (here: temperatures) using our chosen colormap. In our example, the `vmin` temperature (and lower temperatures) will be displayed in purple and the `vmax` temperature (and higher temperatures) will be displayed in red. 

If we narrow the temperature range, temperature differences will be highlighted:

In [ ]:
im = plt.imshow(t, cmap=cmap, vmin=22.5, vmax=23.5)  # vmin and vmax represent the temperatures displayed as purple and red, respectively
plt.colorbar(im, label='Temperature (Celsius)')

Since the temperature array `t` is simply a two-dimensional NumPy array, we can modify the temperature values using all available mathematical operators. Furthermore, temperature values can be extracted using indexing and slicing.

## Quantitative Analysis

We extract a rectangular area between the signal markers with slicing and compute the median temperature in that area.

In [ ]:
area = t[100:300, 100:500]  # slice area (mind the order of coordinates here [y_min:y_max, x_min, x_max])
np.median(area)  # compute the median temperature

**Exercise**: Identify a homogeneous area (a single brick) close to the center of the image and extract that area using slicing. For that area, compute the mean (`np.mean()`) surface temperature and the corresponding NETD noise level as the standard deviation (`np.std()`) of the temperature values in this area.

In [ ]:
# use this cell for the exercise

Now we will generate a horizontal temperature profile across the entire image width. We can extract a horizontal profile with slicing by simply extracting a single row. We pick a row that is not affected by any signal marker. 

In [ ]:
h_profile = t[280,:]  # we slice all columns for row 280

# we display the pixel values (temperatures) in a line plot
f, ax = plt.subplots()
ax.plot(h_profile)
ax.set_xlabel('x Coordinate') 
ax.set_ylabel('Temperature (°C)')

The line profile shows variations through noise and image features (e.g., the brick gaps at x-coordinates 90, 260, 440 and 610), as well as a general gradient. 

## Matching Images

We will now project both images into the same coordinate frame to see whether there are significant differences in their thermal signature. Before we do so, we display both images next to each other. Since we will have to read coordinates for the signal markers, we will choose a denser tick-level and add a grid.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 25))  # create a composite plot with two axes next to each other

# plot image 1
t = thermogram_1.celsius  # temperature data for image 1 in degrees Celsius
ax[0].imshow(t, cmap=cmap)
ax[0].set_title('Image 1')
ax[0].set_xticks(np.arange(0, t.shape[1], 50))  # modify x ticks 
ax[0].set_yticks(np.arange(0, t.shape[0], 50))  # modify y ticks
ax[0].grid()  # add a grid

# plot image 2
t = thermogram_2.celsius
ax[1].imshow(t, cmap=cmap)
ax[1].set_title('Image 2')
ax[1].set_xticks(np.arange(0, t.shape[1], 50))
ax[1].set_yticks(np.arange(0, t.shape[0], 50))
ax[1].grid()


Both images appear to use slightly different temperature ranges. Nevertheless, the signal markers are easy to find in each image.

**Exercise**: Extract the positions of all four circular markers in both images and fill them into the arrays below. Be careful to follow the exact schema as shown below.  

In [ ]:
# use the following schema:
# coo = np.array([(y top left, x top left), (y top right, x top right), (y bottom right, x bottom right), (y bottom left, x bottom left)])

coo_1 = np.array([]) # marker positions for image 1, fill coordinates here!
coo_2 = np.array([]) # marker positions for image 2, fill coordinates here!

We will now plot the extracted coordinates on both images (only possible after completing the corresponding exercise). Take this opportunity to correct the coordinates, if necessary.  

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 25))

# plot image 1
t = thermogram_1.celsius
ax[0].imshow(t, cmap=cmap)
ax[0].set_title('Image 1')
ax[0].scatter(coo_1[:, 0], coo_1[:, 1], c='black')

# plot image 2
t = thermogram_2.celsius
ax[1].imshow(t, cmap=cmap)
ax[1].set_title('Image 2')
ax[1].scatter(coo_2[:, 0], coo_2[:, 1], c='black')


Once the correct coordinates have been extracted, we can transform the second thermogram into the reference frame of the first thermogram. We will use the `ProjectiveTransform` class from `skimage.transform` for this purpose. This class enables a full projective reprojection of the second image on the first image. Be aware that the reprojection only provides a useful result if the marker coordinates have been chosen properly. 

In [ ]:
tform = transform.ProjectiveTransform()  # instantiate projective transformation class
tform.estimate(coo_1, coo_2)  # estimate projective transformation paramterers 
t2_warped = transform.warp(t, tform, output_shape=t.shape, mode='reflect')  # perform reprojection on original temperature data from thermogram_2 

We can now display the warped version of the second thermogram:

In [ ]:
im = plt.imshow(t2_warped, vmin=22, vmax=26, cmap=cmap)
plt.colorbar(im, label='Temperature (Celsius)')

This looks familiar and indeed resembles the visualization of the first thermogram very much.

We will now perform a quantitative comparison of the first thermogram and the warped second thermogram to check for differences. To do so, we compute the pixel-wise difference between the first thermogram and the warped second thermogram: 

In [ ]:
t1 = thermogram_1.celsius
diff = t1-t2_warped  # compute pixel-wise difference

We plot the temperature differences:

In [ ]:
im = plt.imshow(diff, vmin=-1, vmax=1, cmap=cmap)
plt.colorbar(im, label='Temperature Difference (Kelvin)')

The temperature differences are rather homogeneous and close to zero in most locations - except for the signal markers, which serve as reflectors and show different temperatures.

**Exercise**: Quantify the temperature difference by creating (1) one horizontal line profile and (2) one vertical line profile across the temperature difference map. Avoid areas that fall on signal markers. For each profile, compute the mean temperature difference and the corresponding standard deviation.

In [ ]:
# use this cell for the exercise

## Adjusting Camera Parameters

We have seen above that we can access the camera parameters, such as `emissivity`, `object_distance`, `atmospheric_temperature`, `reflected_apparent_temperature` or `relative_humidity`, that were used at the time of observation.

We will now see that we can modify these parameters in the post processing of the data.

To do so, we simply have to use the `adjust_metadata` method on the parameter in question and retrieve the new temperature data from the modified thermogram.

In [ ]:
thermogram_1_mod = thermogram_1.adjust_metadata(emissivity=0.7)  # modify the emissivity in the thermogram
t1_mod = thermogram_1_mod.celsius  # retrieve modified temperature data
np.mean(t1 - t1_mod)  # compute mean temperature difference between original and modified thermogram

**Exercise**: Play with the parameters `emissivity`, `atmospheric_temperature`, and `reflected_apparent_temperature`. How does an increase/decrease in these parameters affect the temperature distribution? 

## Saving Images to File

Finally, we can save our thermograms to file. To do so, we generate a rendering of our data using a specific color palette and temperature range (`min_v` and `max_v`) and then use the `save` method to write it to file. Note that as a result, the new image file will be stripped of its raw data. This means that we cannot read in this new file with the *flyr* module.

In [ ]:
t_pil = thermogram_1.render_pil(unit='celsius', palette='jet', min_v=22, max_v=26).save("test.png")
